Amb els paths predits per tester i guardats a test_paths.txt, podem mesurar les llargades dels paths utilitzant els grafs originals. Així podrem comparar de manera fiable amb les solucions ja conegudes. També ens permet visualitzar el procés de decisió.

# Importacions

In [68]:
!pip install pandas

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 7.3 MB/s eta 0:00:00a 0:00:01


In [69]:
import tarfile
from pathlib import Path
import gzip
import shutil
import tsplib95
import networkx as nx
import re
import pandas as pd

# Càrrega de fitxers

In [59]:
# Rutes
tar_path = Path('Datasets/ALL_tsp.tar')
extract_path = Path('Datasets/ALL_tsp')

# 0. Si la carpeta existeix, esborra tot el contingut
if extract_path.exists():
    shutil.rmtree(extract_path)  # Esborra la carpeta i TOT el que conté

# 1. Crear la carpeta d'extracció
extract_path.mkdir(parents=True, exist_ok=True)

# 2. Obrir i extreure el fitxer .tar
with tarfile.open(tar_path, 'r') as tar:
    tar.extractall(path=extract_path)

# 3. Buscar i descomprimir tots els .gz
gz_files = list(extract_path.glob('*.gz'))

for gz_file in gz_files:
    output_file = extract_path / gz_file.stem  # Elimina ".gz"

    # Descomprimir
    with gzip.open(gz_file, 'rb') as f_in:
        with open(output_file, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)

    # Eliminar el .gz
    gz_file.unlink()

print("ALL_tsp.tar descomprimit i fitxers .gz eliminats.")

ALL_tsp.tar descomprimit i fitxers .gz eliminats.


In [60]:
# Llista pels grafs de test
graphs = []

# Carregar fitxers .tsp
tsp_files = list(extract_path.glob('*.tsp'))

for tsp_file in sorted(tsp_files)[:5]: # -- Limit per no crash Kernel --
    graph = tsplib95.load(tsp_file).get_graph()
    graphs.append(graph)
    print(f"Carregat: {graph.name}")

Carregat: a280
Carregat: ali535
Carregat: att48
Carregat: att532
Carregat: bayg29


In [61]:
# Neteja de fitxers temporals
shutil.rmtree(extract_path)

In [62]:
best_solutions = {}

with open(Path("Datasets/BestSolutionsKnown.txt")) as f:
    for line in f:
        name, rest = line.strip().split(":", 1)     # split només 1 cop
        match = re.search(r"\d+", rest)             # primer enter
        if match:
            best_solutions[name.strip()] = int(match.group(0))

# Funcions

In [63]:
def read_tours(file_path):
    tours = {}
    with open(file_path, "r") as f:
        for line in f:
            name, tour_str = line.strip().split(" : ")
            tour_str = tour_str.strip("[]").replace(" ", "")
            tour_list = [int(x) for x in tour_str.split(",")]
            tours[name] = tour_list
    return tours

In [64]:
def path_length(path, graph):
    total = 0
    for i in range(len(path) - 1):
        u = path[i]
        v = path[i+1]
        total += graph[u][v]['weight']
    total += graph[path[-1]][path[0]]['weight']
    return total

# Avaluació

In [65]:
file_paths = {
    "GNN": Path("Datasets/test_paths.txt"),
    "2-opt": Path("Datasets/tours_2opt.txt"),
    "Christofides": Path("Datasets/tours_christofides.txt"),
    "Greedy": Path("Datasets/tours_greedy.txt"),
    "NN": Path("Datasets/tours_nn.txt"),
    "SA": Path("Datasets/tours_sa.txt"),
    "TA": Path("Datasets/tours_ta.txt")
}

loaded_tours = {
    method: read_tours(path) if path.exists() else {}
    for method, path in file_paths.items()
}

In [ ]:
results = {}

for g in graphs:
    name = g.name
    results[name] = {}

    results[name]["Best"] = best_solutions.get(name, None)

    for method in file_paths.keys():
        if name in loaded_tours[method]:
            tour = loaded_tours[method][name]
            results[name][method] = path_length(tour, g)
        else:
            results[name][method] = None

In [70]:
df = pd.DataFrame.from_dict(results, orient="index")
df

,Best,GNN,2-opt,Christofides,Greedy,NN,SA,TA
a280,2579,4067.0,2838,2923,3157,3157,3157,3157
ali535,202339,NaN,234414,225134,253127,253127,253127,253127
att48,10628,NaN,10959,12613,12861,12861,12861,12861
att532,27686,61155.0,31606,31219,35516,35516,35516,35516
bayg29,1610,3077.0,1762,1716,2005,2005,2005,2005
